# **Environment Setup**

In [1]:
!pip install wordcloud

In [2]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
from collections import Counter

In [3]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Data Pre-processing

In [4]:
movie_df = pd.read_csv('../data/IMDb movies.csv') 
#movie_df_cudf_pd=movie_df_cudf.to_pandas()
movie_df.head()

C:\Users\QiZheng\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [5]:
movie_df.dtypes

imdb_title_id             object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

In [6]:
reduced_df = movie_df[['original_title', 'year', 'date_published','language',
       'genre', 'duration', 'country', 'director', 'writer',
       'production_company', 'actors', 'avg_vote', 'votes',
       'budget', 'worlwide_gross_income']]
reduced_df

,original_title,year,date_published,language,genre,duration,country,director,writer,production_company,actors,avg_vote,votes,budget,worlwide_gross_income
0,Miss Jerry,1894,1894-10-09,None,Romance,45,USA,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9,154,NaN,NaN
1,The Story of the Kelly Gang,1906,1906-12-26,None,"Biography, Crime, Drama",70,Australia,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1,589,$ 2250,NaN
2,Den sorte drøm,1911,1911-08-19,NaN,Drama,53,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8,188,NaN,NaN
3,Cleopatra,1912,1912-11-13,English,"Drama, History",100,USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2,446,$ 45000,NaN
4,L'Inferno,1911,1911-03-06,Italian,"Adventure, Drama, Fantasy",68,Italy,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0,2237,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,Le lion,2020,2020-01-29,French,Comedy,95,"France, Belgium",Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour",Monkey Pack Films,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",5.3,398,NaN,$ 3507171
85851,De Beentjes van Sint-Hildegard,2020,2020-02-13,"German, Dutch","Comedy, Drama",103,Netherlands,Johan Nijenhuis,"Radek Bajgar, Herman Finkers",Johan Nijenhuis & Co,"Herman Finkers, Johanna ter Steege, Leonie ter...",7.7,724,NaN,$ 7299062
85852,Padmavyuhathile Abhimanyu,2019,2019-03-08,Malayalam,Drama,130,India,Vineesh Aaradya,"Vineesh Aaradya, Vineesh Aaradya",RMCC Productions,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",7.9,265,NaN,NaN
85853,Sokagin Çocuklari,2019,2019-03-15,Turkish,"Drama, Family",98,Turkey,Ahmet Faik Akinci,"Ahmet Faik Akinci, Kasim Uçkan",Gizem Ajans,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",6.4,194,NaN,$ 2833


In [7]:
len(reduced_df)

85855

In [8]:
clean_movie_df = reduced_df.dropna()

In [9]:
sample_df=clean_movie_df[clean_movie_df["budget"].str.startswith("$")]
sample_df=sample_df[clean_movie_df["worlwide_gross_income"].str.startswith("$")]
sample_df['budget'] = sample_df['budget'].str.replace('$','')
sample_df['worlwide_gross_income'] = sample_df['worlwide_gross_income'].str.replace('$','')
sample_df['budget'] = sample_df['budget'].astype(np.int64)
sample_df['worlwide_gross_income'] = sample_df['worlwide_gross_income'].astype(np.int64)
sample_df['year'] = sample_df['year'].astype(np.int64)
sample_df

<ipython-input-9-7ac5aebce545>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sample_df=sample_df[clean_movie_df["worlwide_gross_income"].str.startswith("$")]


,original_title,year,date_published,language,genre,duration,country,director,writer,production_company,actors,avg_vote,votes,budget,worlwide_gross_income
165,Das Cabinet des Dr. Caligari,1920,1920-02-27,German,"Fantasy, Horror, Mystery",76,Germany,Robert Wiene,"Carl Mayer, Hans Janowitz",Decla-Bioscop AG,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",8.1,55601,18000,8811
245,The Four Horsemen of the Apocalypse,1921,1923-04-16,None,"Drama, Romance, War",150,USA,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",7.2,3058,800000,9183673
251,The Kid,1921,1923-11-26,"English, None","Comedy, Drama, Family",68,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Carl Miller, Edna Purviance, Jackie Coogan, Ch...",8.3,109038,250000,26916
348,A Woman of Paris: A Drama of Fate,1923,1927-06-06,"None, English","Drama, Romance",82,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Clarence Geldart, Carl Miller,...",7.0,4735,351000,11233
415,The Gold Rush,1925,1925-10-23,"English, None","Adventure, Comedy, Drama",95,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Mack Swain, Tom Murray, Henry...",8.2,97480,923000,26916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85477,Plane Tree,2018,2019-04-12,Turkish,Drama,99,Turkey,Mustafa Karadeniz,"Necip Güleçer, Mustafa Karadeniz",Cinar Film Yapim,"Sezgin Cengiz, Silan Düzdaban, Yunus Emre Çeli...",6.8,168,200000,4183
85530,Doblemente Embarazada,2019,2019-12-20,Spanish,Comedy,98,Mexico,Koko Stambuk,"Diego Ayala, Koko Stambuk",SDB Films,"Roberta Burns, Gustavo Egelhaaf, Luis Felipe C...",4.4,197,950000,965558
85592,Quezon's Game,2018,2019-05-29,English,"Drama, History",127,Philippines,Matthew E. Rosen,"Janice Y. Perez, Dean Rosen",Star Cinema,"Raymond Bagatsing, Rachel Alejandro, Kate Alej...",7.5,117,500000,5986
85740,I Still Believe,2020,2020-03-19,English,"Biography, Drama, Music",116,USA,"Andrew Erwin, Jon Erwin","Jeremy Camp, Jon Erwin",Kevin Downes Productions,"K.J. Apa, Britt Robertson, Nathan Parsons, Gar...",6.5,6196,12000000,13681524


In [10]:
sample_df.dtypes

original_title            object
year                       int64
date_published            object
language                  object
genre                     object
duration                   int64
country                   object
director                  object
writer                    object
production_company        object
actors                    object
avg_vote                 float64
votes                      int64
budget                     int64
worlwide_gross_income      int64
dtype: object

In [11]:
def extract_day(film_date):
    try:
        datetime_object = datetime.strptime(film_date, '%Y-%m-%d')
        film_day = datetime_object.timetuple().tm_yday
        return film_day
    except:
        return 0

In [12]:
sample_df['day']=sample_df['date_published'].apply(lambda x: extract_day(x))
sample_df=sample_df[sample_df['day']>0]
sample_df

,original_title,year,date_published,language,genre,duration,country,director,writer,production_company,actors,avg_vote,votes,budget,worlwide_gross_income,day
165,Das Cabinet des Dr. Caligari,1920,1920-02-27,German,"Fantasy, Horror, Mystery",76,Germany,Robert Wiene,"Carl Mayer, Hans Janowitz",Decla-Bioscop AG,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",8.1,55601,18000,8811,58
245,The Four Horsemen of the Apocalypse,1921,1923-04-16,None,"Drama, Romance, War",150,USA,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",7.2,3058,800000,9183673,106
251,The Kid,1921,1923-11-26,"English, None","Comedy, Drama, Family",68,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Carl Miller, Edna Purviance, Jackie Coogan, Ch...",8.3,109038,250000,26916,330
348,A Woman of Paris: A Drama of Fate,1923,1927-06-06,"None, English","Drama, Romance",82,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Clarence Geldart, Carl Miller,...",7.0,4735,351000,11233,157
415,The Gold Rush,1925,1925-10-23,"English, None","Adventure, Comedy, Drama",95,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Mack Swain, Tom Murray, Henry...",8.2,97480,923000,26916,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85477,Plane Tree,2018,2019-04-12,Turkish,Drama,99,Turkey,Mustafa Karadeniz,"Necip Güleçer, Mustafa Karadeniz",Cinar Film Yapim,"Sezgin Cengiz, Silan Düzdaban, Yunus Emre Çeli...",6.8,168,200000,4183,102
85530,Doblemente Embarazada,2019,2019-12-20,Spanish,Comedy,98,Mexico,Koko Stambuk,"Diego Ayala, Koko Stambuk",SDB Films,"Roberta Burns, Gustavo Egelhaaf, Luis Felipe C...",4.4,197,950000,965558,354
85592,Quezon's Game,2018,2019-05-29,English,"Drama, History",127,Philippines,Matthew E. Rosen,"Janice Y. Perez, Dean Rosen",Star Cinema,"Raymond Bagatsing, Rachel Alejandro, Kate Alej...",7.5,117,500000,5986,149
85740,I Still Believe,2020,2020-03-19,English,"Biography, Drama, Music",116,USA,"Andrew Erwin, Jon Erwin","Jeremy Camp, Jon Erwin",Kevin Downes Productions,"K.J. Apa, Britt Robertson, Nathan Parsons, Gar...",6.5,6196,12000000,13681524,79


In [13]:
sample_df=sample_df.drop(['date_published'],axis=1)

In [14]:
list_of_languages = list(sample_df['language'].apply(lambda x:  [i for i in x.split(', ')]).values)
top_languages = [x[0] for x in Counter([i for j in list_of_languages for i in j]).most_common()[:30]]

In [15]:
list_of_genres = list(sample_df['genre'].apply(lambda x:  [i for i in x.split(', ')]).values)
all_genres = [x[0] for x in Counter([i for j in list_of_genres for i in j]).most_common()]

In [16]:
list_of_countries = list(sample_df['country'].apply(lambda x:  [i for i in x.split(', ')]).values)
top_countries = [x[0] for x in Counter([i for j in list_of_countries for i in j]).most_common()[:30]]

In [17]:
list_of_actors = list(sample_df['actors'].apply(lambda x:  [i for i in x.split(', ')]).values)
rd_list_of_actors=[x[0:5] for x in list_of_actors]
all_actors = Counter([i for j in rd_list_of_actors for i in j]).most_common()
top_actors = [x[0] for x in all_actors if x[1]>10]

In [18]:
list_of_directors = list(sample_df['director'].apply(lambda x:  [i for i in x.split(', ')]).values)
all_directors = Counter([i for j in list_of_directors for i in j]).most_common()
top_directors = [x[0] for x in all_directors if x[1]>=5]

In [19]:
list_of_writers = list(sample_df['writer'].apply(lambda x:  [i for i in x.split(', ')]).values)
all_writers = Counter([i for j in list_of_writers for i in j]).most_common()
top_writers = [x[0] for x in all_writers if x[1]>=5]

In [20]:
list_of_companies = list(sample_df['production_company'].apply(lambda x:  [i for i in x.split(', ')]).values)
all_companies = Counter([i for j in list_of_companies for i in j]).most_common()
top_companies = [x[0] for x in all_companies if x[1]>=5]

In [21]:
for language in top_languages:
  sample_df['language_' + language] = sample_df['language'].apply(lambda x: 1 if language in x.split(', ') else 0)

In [22]:
for genre in all_genres:
  sample_df['genre_' + genre] = sample_df['genre'].apply(lambda x: 1 if genre in x.split(', ') else 0)

In [23]:
for country in top_countries:
  sample_df['country_' + country] = sample_df['country'].apply(lambda x: 1 if country in x.split(', ') else 0)

In [24]:
for director in top_directors:
  sample_df['director_' + director] = sample_df['director'].apply(lambda x: 1 if director in x.split(', ') else 0)

In [25]:
for writer in top_writers:
  sample_df['writer_' + writer] = sample_df['writer'].apply(lambda x: 1 if writer in x.split(', ') else 0)

In [26]:
for company in top_companies:
  sample_df['company_' + company] = sample_df['production_company'].apply(lambda x: 1 if company in x.split(', ') else 0)

In [27]:
for actor in top_actors:
  sample_df['actor_' + actor] = sample_df['actors'].apply(lambda x: 1 if actor in x.split(', ')[0:5] else 0)

In [28]:
sample_df=sample_df.reset_index(drop=True)

In [29]:
sample_df=sample_df.select_dtypes(include=['int64','float64'])

In [30]:
sample_df.shape

(8870, 1872)

In [31]:
sample_df.head()

,year,duration,avg_vote,votes,budget,worlwide_gross_income,day,language_English,language_Spanish,language_French,...,actor_Matthew Goode,actor_Edgar Ramírez,actor_Tyler Perry,actor_Sebastian Stan,actor_Rebecca Hall,actor_Cam Gigandet,actor_Miles Teller,actor_Scott Adkins,actor_Octavia Spencer,actor_Dave Bautista
0,1920,76,8.1,55601,18000,8811,58,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1921,150,7.2,3058,800000,9183673,106,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1921,68,8.3,109038,250000,26916,330,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1923,82,7.0,4735,351000,11233,157,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1925,95,8.2,97480,923000,26916,296,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
sample_df.to_csv("encoded_data/sample_data.csv", index=False)

In [ ]:
#plt.figure(figsize = (12, 8))
#text = ' '.join([i for j in list_of_genres for i in j])
#wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
#                      width=1200, height=1000).generate(text)
#plt.imshow(wordcloud)
#plt.title('Top genres')
#plt.axis("off")
#plt.show()

# Model test

In [ ]:
sample_df.head()

In [ ]:
X = sample_df.drop(["avg_vote","worlwide_gross_income"], axis=1)

In [ ]:
X.head()

In [ ]:
vote_y=sample_df["avg_vote"]
nvote_y=sample_df["votes"]
revenue_y=sample_df["worlwide_gross_income"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, revenue_y_train, revenue_y_test = train_test_split(X, revenue_y, test_size=0.2)

In [ ]:
X_train.head()

# Scale the duration and day columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
duration_scaler=MinMaxScaler().fit(X[["duration"]])
day_scaler=MinMaxScaler().fit(X[["day"]])

In [ ]:
X_train_scaled=X_train
X_test_scaled=X_test
revenue_y_train_scaled=revenue_y_train
revenue_y_test_scaled=revenue_y_test

In [ ]:
X_train_scaled["duration"]=duration_scaler.transform(X_train_scaled[["duration"]])
X_train_scaled["day"]=day_scaler.transform(X_train_scaled[["day"]])

X_test_scaled["duration"]=duration_scaler.transform(X_test_scaled[["duration"]])
X_test_scaled["day"]=day_scaler.transform(X_test_scaled[["day"]])

In [ ]:
print('X_train dimensions: ', X_train_scaled.shape, 'y_train: ', revenue_y_train_scaled.shape)
print('X_test dimensions:', X_test_scaled.shape, 'y_validation: ', revenue_y_test_scaled.shape)

In [ ]:
from xgboost import XGBRegressor
regressor = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 5, min_child_weight= 5,
                         subsample = 0.8, objective='reg:squarederror')
regressor.fit(X_train_scaled, revenue_y_train_scaled)

In [ ]:
revenue_y_pred = regressor.predict(X_test_scaled)
from sklearn.metrics import r2_score
score = r2_score(revenue_y_test_scaled, revenue_y_pred)
print(score)

In [ ]:
select_features=[]
for col,score in zip(X_train.columns,regressor.feature_importances_):
  if score >0.00001:
    select_features.append(col)
    print(col,score)

In [ ]:
len(select_features)

In [ ]:
select_sample_df=sample_df[select_features]

In [ ]:
select_sample_df

In [ ]:
X_selected = select_sample_df
X_selected_train, X_selected_test, revenue_y_train, revenue_y_test = train_test_split(X_selected, revenue_y, test_size = 0.2)

X_selected_train_scaled=X_selected_train
X_selected_test_scaled=X_selected_test
revenue_y_train_scaled=revenue_y_train
revenue_y_test_scaled=revenue_y_test

X_selected_train_scaled["duration"]=duration_scaler.transform(X_selected_train_scaled[["duration"]])
X_selected_train_scaled["day"]=day_scaler.transform(X_selected_train_scaled[["day"]])
X_selected_test_scaled["duration"]=duration_scaler.transform(X_selected_test_scaled[["duration"]])
X_selected_test_scaled["day"]=day_scaler.transform(X_selected_test_scaled[["day"]])

print('X_train dimensions: ', X_selected_train_scaled.shape, 'y_train: ', revenue_y_train_scaled.shape)
print('X_test dimensions:', X_selected_test_scaled.shape, 'y_validation: ', revenue_y_test_scaled.shape)

In [ ]:
regressor = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 5, min_child_weight= 5,
                         subsample = 0.8, objective='reg:squarederror')
regressor.fit(X_selected_train_scaled, revenue_y_train_scaled)

In [ ]:
revenue_y_pred = regressor.predict(X_selected_test_scaled)
from sklearn.metrics import r2_score
score = r2_score(revenue_y_test_scaled, revenue_y_pred)
print(score)

In [ ]:
regressor = XGBRegressor(max_depth= 7, min_child_weight= 4, learning_rate=0.1,
                         n_estimators=500, colsample_bytree= 0.9, subsample = 0.7,
                         gamma= 0.7, objective='reg:squarederror')

In [ ]:
gridparas = {
#    'max_depth': [3,5, 7],
#    'learning_rate': [0.06,0.1,0.3],
#    'n_estimators': [100,200, 500],
#    'min_child_weight': [2, 3, 4],
#    'max_delta_step': [0, 0.2, 0.6, 1, 2],
#    'subsample': [0.5,0.6,0.7],
#    'colsample_bytree': [0.8,0.9],
#    'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#    'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#    'gamma': [0.1, 0.3],
#    'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
gsearch = GridSearchCV(regressor, param_grid=gridparas,verbose=3, n_jobs=-1)
gsearch.fit(X_selected_train_scaled,revenue_y_train_scaled)

In [ ]:
print(gsearch.best_params_)
print(gsearch.best_score_)

In [ ]:
regressor = XGBRegressor(max_depth= 7, min_child_weight= 4, learning_rate=0.1,
                         n_estimators=500, colsample_bytree= 0.9, subsample = 0.7,
                         gamma= 0.7, objective='reg:squarederror')
regressor.fit(X_selected_train_scaled, revenue_y_train_scaled)

In [ ]:
revenue_y_pred = regressor.predict(X_selected_test_scaled)
from sklearn.metrics import r2_score
score = r2_score(revenue_y_test_scaled, revenue_y_pred)
print(score)

In [ ]:
import joblib
filename = 'Model/xgboost_model.sav'
joblib.dump(regressor, filename)

In [ ]:
filename = 'Model/xgboost_model.h5'
joblib.dump(regressor, filename)

In [ ]:
regressor.save_model("Model/xgboost_regressor.sav")

In [ ]:
select_sample_df.to_csv("Encoded_data/select_sample_df_xgb.csv",index=False)

In [ ]:
revenue_y.to_csv("Encoded_data/revenue_xgb.csv",index=False)